***Esse documento é para ser usado somente para ver como a IA funciona, para entender a lógica e a construção por trás da IA veja o README.md e o arquivo Detector (Lógica)***

In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
mp_holistico = mp.solutions.holistic
mp_desenho = mp.solutions.drawing_utils

In [3]:
def deteccao_mediapipe(imagem, modelo):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem.flags.writeable = False
    resultados = modelo.process(imagem)
    imagem.flags.writeable = True
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR)
    return imagem, resultados

In [4]:
def desenhar_landmarks(imagem, resultados):
    # Desenhar as conexões do rosto
    mp_desenho.draw_landmarks(imagem, resultados.face_landmarks, mp_holistico.FACEMESH_CONTOURS,
                             mp_desenho.DrawingSpec(color=(80, 110 ,10), thickness=1, circle_radius=1),
                             mp_desenho.DrawingSpec(color=(80, 255, 121), thickness=1, circle_radius=1))
    
    # Desenhar as conexões do tronco
    mp_desenho.draw_landmarks(imagem, resultados.pose_landmarks, mp_holistico.POSE_CONNECTIONS,
                             mp_desenho.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_desenho.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    
    # Desenhar as conexões da mão esquerda
    mp_desenho.draw_landmarks(imagem, resultados.left_hand_landmarks, mp_holistico.HAND_CONNECTIONS,
                             mp_desenho.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_desenho.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    
    # Desenhar as conexões da mão direita
    mp_desenho.draw_landmarks(imagem, resultados.right_hand_landmarks, mp_holistico.HAND_CONNECTIONS,
                             mp_desenho.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_desenho.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [5]:
def extrair_pontos_chaves(resultados):
    # Condição para pose_landmarks
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in resultados.pose_landmarks.landmark]).flatten() if resultados.pose_landmarks else np.zeros(132)

    # Condição para face_landmarks
    rosto = np.array([[res.x, res.y, res.z] for res in resultados.face_landmarks.landmark]).flatten() if resultados.face_landmarks else np.zeros(1404)

    # Condição para left_hand_landmarks
    esquerda = np.array([[res.x, res.y, res.z] for res in resultados.left_hand_landmarks.landmark]).flatten() if resultados.left_hand_landmarks else np.zeros(63)


    # Condição para right_hand_landmarks
    direita = np.array([[res.x, res.y, res.z] for res in resultados.right_hand_landmarks.landmark]).flatten() if resultados.right_hand_landmarks else np.zeros(63)

    return np.concatenate([pose, rosto, esquerda, direita])

In [6]:
# Caminho para exportar os dados, arrays NumPy
CAMINHO_DADOS = os.path.join('Dados_MP')

# Ações que queremos que sejam detecdadas
acoes = np.array(['ola', 'obrigado', 'amo voce'])

# Quantidade de sequências de vídeos que queremos capturar
num_videos = 30

# Quantidade de frames capturadas
num_frames = 30

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
mapa_rotulos = {rotulo: num for num, rotulo in enumerate(acoes)}

In [9]:
videos, rotulos = [], []
for acao in acoes:
    for video in range(num_videos):
        tela = []
        for frame_num in range(num_frames):
            res = np.load(os.path.join(CAMINHO_DADOS, acao, str(video), f'{frame_num}.npy'))
            tela.append(res)
        videos.append(tela)
        rotulos.append(mapa_rotulos[acao])

In [10]:
X = np.array(videos)
y = to_categorical(rotulos).astype(int)

In [11]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.05)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
modelo = Sequential()
modelo.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
modelo.add(LSTM(128, return_sequences=True, activation='relu'))
modelo.add(LSTM(64, return_sequences=False, activation='relu'))

modelo.add(Dense(64, activation='relu'))
modelo.add(Dense(32, activation='relu'))
modelo.add(Dense(acoes.shape[0], activation='softmax'))

In [14]:
modelo.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [15]:
modelo.load_weights('modelo_acao.h5')

In [16]:
cores = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def visuzalizar_probabilidades(res, acoes, frame_entrada, cores):
    frame_saida = frame_entrada.copy()
    for num, probabilidade in enumerate(res):
        cv2.rectangle(frame_saida, (0, 60+num*40), (int(probabilidade*100), 90+num*40), cores[num], -1)
        cv2.putText(frame_saida, acoes[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return frame_saida

In [17]:
# 1. Novas variáveis de detecção
sequencias = []
sentencas = []
previsoes = []
limite = 0.7

cap = cv2.VideoCapture(0)

# Iniciar o modelo mediapipe
with mp_holistico.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistico:
    while cap.isOpened():
        # Ler o que a câmera está captando
        ret, frame = cap.read()

        # Fazer as detecções
        imagem, resultados = deteccao_mediapipe(frame, holistico)
        
        # Desenhar landmarks
        desenhar_landmarks(imagem, resultados)
        
        # 2. Lógica de previsão
        pontos_chaves = extrair_pontos_chaves(resultados)
        sequencias.append(pontos_chaves)
        sequencia = sequencias[-30:]
        
        if len(sequencia) == 30:
            res = modelo.predict(np.expand_dims(sequencia, axis=0))[0]
            previsoes.append(np.argmax(res))
            
        # 3. Lógica de visualização
            if np.unique(previsoes[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > limite:
                    if len(sentencas) > 0:
                        if acoes[np.argmax(res)] != sentencas[-1]:
                            sentencas.append(acoes[np.argmax(res)])
                    else:
                        sentencas.append(acoes[np.argmax(res)])

            if len(sentencas) > 5:
                sentencas = sentencas[-5:]

            # Visualizar da forma com probabilidade as ações
            imagem = visuzalizar_probabilidades(res, acoes, imagem, cores)
            
        # Visualizar da forma escrita as ações
        cv2.rectangle(imagem, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(imagem, ' '.join(sentencas), (3, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            

        # Mostrar na tela
        cv2.imshow('Tela OpenCV', imagem)

        # Encerrar o loop
        if cv2.waitKey(10) & 0XFF == ord('s'):
            break
    cap.release()
    cv2.destroyAllWindows()